In [2]:
# Import libraries
import matplotlib.pyplot as plt # Matplotlib is used for plotting
import pandas as pd             # Pandas helps organize data structures
import numpy as np              # Numpy provides mathematical functions like sqrt()
import statsmodels.formula.api as sm  # Statsmodel provides linear regression
from sklearn import metrics 
from sklearn.model_selection import KFold
import skimage.io as io # Loads and displays images
from scipy.linalg import svd
from pycocotools.coco import COCO # Coco dataset


ModuleNotFoundError: No module named 'pycocotools'

In [10]:
dataDir=''
dataType='val2017'
annFile='./annotations/captions_{}.json'.format(dataType)

coco = COCO(annFile)

loading annotations into memory...


FileNotFoundError: ignored

In [0]:
# load and display image
# I = io.imread('%s/images/%s/%s'%(dataDir,dataType,img['file_name']))


In [0]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms))

1. Convert data to dataframe with form: 

image url, apple, orange, carrot, brocolli

image1.png, 0, 1, 0, 0, ...


2. (For Efficiency) Do PCA on all images to reduce # of parameters


3. Use K-Fold model selection with k=10 to pick best model


4. Try on some photos of our own